In [48]:
# !pip install -r ../requirements.txt
# !pip3 install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu

In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import uproot
import time

# import torch

In [50]:
file_path = "../../sim_first/build/plot_1e7.root"
file = uproot.open(file_path)
time_ns = file['Photons']['fTime'].array(library='np')

Note that we can't convert this numpy array (which is in float64) to float32 without compromising the timing accuracy, and the MPS backend doesn't support float64. So, we will sort in numpy and get the time coincidence pairs, and only then do tensor stuff.

In [51]:
time_ns_sorted_index = np.argsort(time_ns)

In [52]:
time_ns = time_ns[time_ns_sorted_index]

In [53]:
def time_window_filter(time_ns : np.array, window_ns):
    coincidences = []
    for i in range(len(time_ns)):
        temp_coincidences = []
        j = i + 1
        while j < len(time_ns) and abs(time_ns[i] - time_ns[j]) < window_ns:
            temp_coincidences.append((i, j))
            j += 1
        coincidences += temp_coincidences
    return coincidences

### Code for a Basic Timing Check

In [54]:
# time_window_coincidences = time_window_filter(time_ns, 4)

In [55]:
# time_window_coincidences = np.array(time_window_coincidences)


That was pretty slow; and I don't think we can do much better unless we use Cython or something.
A simple for loop takes 30 seconds:

In [56]:
# temp = 0
# for i in range(len(time_ns)):
#     temp += time_ns[i] 

We have now got the indices that will sort the corresponding columns, and the timing coincidence (n, 2) list of pairs. Time to start batching!

In [57]:
def load_data_and_slice(root_file, col_name, start, end):
    print(f"Function Begins: Loading {col_name} from {start} to {end}")
    start_time = time.time()
    print("Loading Data...")
    data = root_file['Photons'][col_name].array(library='np')
    print("Data Loaded. Took {} seconds".format(time.time() - start_time))
    print("Sorting Data...")
    data = data[time_ns_sorted_index]
    print("Data Sorted. Took {} seconds".format(time.time() - start_time))
    print("Slicing Data...")
    data = data[start:end]
    print("Data Sliced. Took {} seconds".format(time.time() - start_time))

    # print("Converting Numpy Array Datatype")
    # if col_name == 'fEventID':
    #     data = data.astype('int64')
    # else:
    #     data = data.astype('float32')
    # print("Data Conversion Complete. Took {} seconds".format(time.time() - start_time))

    # print("Converting Numpy Array to Torch Tensor")
    # data = torch.from_numpy(data).to(DEVICE)
    # print("Data Conversion Complete. Took {} seconds".format(time.time() - start_time))
    return data

In [58]:
# eventID_0_10000 = load_data_and_slice(file, 'fEvent', 0, 10000)

In [59]:
def angle_between_vectors(v1, v2):
    #v1 and v2 are arrays of shape (3,)
    #returns a float

    return np.arccos(np.sum(v1*v2)/(np.linalg.norm(v1)*np.linalg.norm(v2)))


def vectorized_angle_between_vectors(v1_array, v2_array):
    #v1 and v2 are arrays of shape (n, 3)
    #returns an array of shape (n, 1)

    return np.arccos(np.sum(v1_array*v2_array, axis=1)/(np.linalg.norm(v1_array, axis=1)*np.linalg.norm(v2_array, axis=1)))

def vectorized_normaldotprod_between_vectors(v1_array, v2_array):
    #v1 and v2 are arrays of shape (n, 3)
    #returns an array of shape (n, 1)

    return np.sum(v1_array*v2_array, axis=1)/(np.linalg.norm(v1_array, axis=1)*np.linalg.norm(v2_array, axis=1))

def energy_filter(coincidences, energy_keV_blurred, energy_keV_min, energy_keV_max):
    #so the coincidence array has shape (n, 2)
    #we want to filter out the rows where the energy of both events is not in the specified range

    #we will do this by first creating a boolean array of shape (n, 2) where the value is True if the energy is in the specified range
    #and then we will check if both values in the row are True
    #this will give us a boolean array of shape (n, 1)
    #we will then use this array to index the coincidence array and get the filtered coincidences

    #create the boolean array
    bool_array = np.array([(energy_keV_min < energy_keV_blurred[coincidences[:, 0]]) \
         & (energy_keV_blurred[coincidences[:, 0]] < energy_keV_max), \
            (energy_keV_min < energy_keV_blurred[coincidences[:, 1]]) \
         & (energy_keV_blurred[coincidences[:, 1]] < energy_keV_max)])
    bool_array = np.transpose(bool_array)

    #check if both values in the row are True
    bool_array = np.all(bool_array, axis=1)

    #index the coincidence array
    filtered_coincidences = coincidences[bool_array]

    return filtered_coincidences

def count_true_in_filtered_coincidences(eventID, coincidences, x_mm, y_mm, z_mm):
    coincidences = np.array(coincidences)
    cond1 = (eventID[coincidences[:, 0]] == eventID[coincidences[:, 1]])
    #condition 2 is that te events should be roughly diametrically opposite
    #we will use the dot product of the polarisation vectors to check this
    #the dot product of two vectors is 1 if they are parallel, -1 if they are anti-parallel and 0 if they are perpendicular
    #so we will check if the dot product is less than -0.9
    v1 = np.array([x_mm[coincidences[:, 0]], y_mm[coincidences[:, 0]], z_mm[coincidences[:, 0]]]).T
    v2 = np.array([x_mm[coincidences[:, 1]], y_mm[coincidences[:, 1]], z_mm[coincidences[:, 1]]]).T
    cond2 = (vectorized_normaldotprod_between_vectors(v1, v2) < -0.9)
    return np.sum(cond1 & cond2)

def polarization_filter(coincidences, pol_angle_diff_blurred, angle_diff_min, angle_diff_max):
    bool_array = (pol_angle_diff_blurred > angle_diff_min) & (pol_angle_diff_blurred < angle_diff_max)
    return coincidences[bool_array]

def full_pipeline( #assume everything sorted wrt time_ns
        time_window_coincidences, 
        energy_keV,
        x_mm,
        y_mm,
        z_mm,
        polX,
        polY,
        polZ,
        eventID,
):
    energy_keV_blurred = np.random.normal(energy_keV, 0.1*energy_keV)
    energy_coincidences = energy_filter(time_window_coincidences, energy_keV_blurred, 350.0, 650.0)
    true_in_energy_filtered_count = count_true_in_filtered_coincidences(eventID, energy_coincidences, x_mm, y_mm, z_mm)
    total_in_energy_filtered_count = len(energy_coincidences)

    polX_1 = polX[time_window_coincidences[:, 0]]
    polY_1 = polY[time_window_coincidences[:, 0]]
    polZ_1 = polZ[time_window_coincidences[:, 0]]

    polX_2 = polX[time_window_coincidences[:, 1]]
    polY_2 = polY[time_window_coincidences[:, 1]]
    polZ_2 = polZ[time_window_coincidences[:, 1]]

    v1 = np.array([polX_1, polY_1, polZ_1]).T
    v2 = np.array([polX_2, polY_2, polZ_2]).T

    pol_angle_diff = vectorized_angle_between_vectors(v1, v2)
    pol_angle_diff_blurred = np.random.normal(pol_angle_diff, 0.1*np.abs(pol_angle_diff))

    polarization_coincidences = polarization_filter(time_window_coincidences, pol_angle_diff_blurred, 1.075, 2.000)
    true_in_polarization_filtered_count = count_true_in_filtered_coincidences(eventID, polarization_coincidences, x_mm, y_mm, z_mm)
    total_in_polarization_filtered_count = len(polarization_coincidences)

    energy_and_polarization_coincidences = energy_filter(polarization_coincidences, energy_keV_blurred, 350.0, 650.0)
    true_in_energy_and_polarization_filtered_count = count_true_in_filtered_coincidences(eventID, energy_and_polarization_coincidences, x_mm, y_mm, z_mm)
    total_in_energy_and_polarization_filtered_count = len(energy_and_polarization_coincidences)

    return len(x_mm), \
    len(time_window_coincidences), \
    true_in_energy_filtered_count, \
    total_in_energy_filtered_count, \
    true_in_polarization_filtered_count, \
    total_in_polarization_filtered_count, \
    true_in_energy_and_polarization_filtered_count, \
    total_in_energy_and_polarization_filtered_count
    
    

In [60]:
def batched_pipeline(batch_count):
    #batch_size, taking into account the last batch
    batch_size = int((len(time_ns) + (batch_count - 1))/batch_count) #ceil division
    cum_array = np.zeros((batch_count, 8), dtype='int64')

    print("Starting Batched Pipeline")
    global_start_time = time.time()
    for i in range(batch_count):
        print("Batch {} of {}".format(i+1, batch_count))
        batch_start_time = time.time()
        start = i*batch_size
        end = min((i+1)*batch_size, len(time_ns))
        time_ns_batch = time_ns[start:end]
        time_window_coincidences_batch = time_window_filter(time_ns_batch, 4) #we WILL be losing coincidences across two batches, but they are small in number
        time_window_coincidences_batch = np.array(time_window_coincidences_batch)
        energy_keV_batch = load_data_and_slice(file, 'fEnergy', start, end)/1000.0 #convert to keV from eV from file
        x_mm_batch = load_data_and_slice(file, 'fX', start, end)
        y_mm_batch = load_data_and_slice(file, 'fY', start, end)
        z_mm_batch = load_data_and_slice(file, 'fZ', start, end)
        polX_batch = load_data_and_slice(file, 'fPolX', start, end)
        polY_batch = load_data_and_slice(file, 'fPolY', start, end)
        polZ_batch = load_data_and_slice(file, 'fPolZ', start, end)
        eventID_batch = load_data_and_slice(file, 'fEvent', start, end)
        
        temp_array = np.array(list(full_pipeline(
            time_window_coincidences_batch, 
            energy_keV_batch,
            x_mm_batch,
            y_mm_batch,
            z_mm_batch,
            polX_batch,
            polY_batch,
            polZ_batch,
            eventID_batch,
        )))

        print(temp_array)
        cum_array[i] = temp_array

        print("Batch {} of {} Complete. Took {} seconds".format(i+1, batch_count, time.time() - batch_start_time))

    print("Batched Pipeline Complete. Took {} seconds".format(time.time() - global_start_time))
    print(cum_array)
    return cum_array

In [61]:
final_array = batched_pipeline(10)

Starting Batched Pipeline
Batch 1 of 10
Function Begins: Loading fEnergy from 0 to 202215
Loading Data...
Data Loaded. Took 0.1347029209136963 seconds
Sorting Data...
Data Sorted. Took 0.1462700366973877 seconds
Slicing Data...
Data Sliced. Took 0.14631104469299316 seconds
Function Begins: Loading fX from 0 to 202215
Loading Data...


Data Loaded. Took 0.24605298042297363 seconds
Sorting Data...
Data Sorted. Took 0.2550680637359619 seconds
Slicing Data...
Data Sliced. Took 0.2551000118255615 seconds
Function Begins: Loading fY from 0 to 202215
Loading Data...
Data Loaded. Took 0.16139602661132812 seconds
Sorting Data...
Data Sorted. Took 0.16968679428100586 seconds
Slicing Data...
Data Sliced. Took 0.16973495483398438 seconds
Function Begins: Loading fZ from 0 to 202215
Loading Data...
Data Loaded. Took 0.13572907447814941 seconds
Sorting Data...
Data Sorted. Took 0.14412212371826172 seconds
Slicing Data...
Data Sliced. Took 0.14415502548217773 seconds
Function Begins: Loading fPolX from 0 to 202215
Loading Data...
Data Loaded. Took 0.21799492835998535 seconds
Sorting Data...
Data Sorted. Took 0.22955584526062012 seconds
Slicing Data...
Data Sliced. Took 0.22958898544311523 seconds
Function Begins: Loading fPolY from 0 to 202215
Loading Data...
Data Loaded. Took 0.15674901008605957 seconds
Sorting Data...
Data Sorte

/var/folders/t0/kz8yxky97yg56bbdw_h8qs100000gn/T/ipykernel_89254/3422969115.py:12: RuntimeWarning: invalid value encountered in divide
  return np.arccos(np.sum(v1_array*v2_array, axis=1)/(np.linalg.norm(v1_array, axis=1)*np.linalg.norm(v2_array, axis=1)))


Function Begins: Loading fEnergy from 202215 to 404430
Loading Data...
Data Loaded. Took 0.07685017585754395 seconds
Sorting Data...
Data Sorted. Took 0.0865020751953125 seconds
Slicing Data...
Data Sliced. Took 0.08653903007507324 seconds
Function Begins: Loading fX from 202215 to 404430
Loading Data...
Data Loaded. Took 0.11656904220581055 seconds
Sorting Data...
Data Sorted. Took 0.12671494483947754 seconds
Slicing Data...
Data Sliced. Took 0.1268758773803711 seconds
Function Begins: Loading fY from 202215 to 404430
Loading Data...
Data Loaded. Took 0.12171077728271484 seconds
Sorting Data...
Data Sorted. Took 0.1312577724456787 seconds
Slicing Data...
Data Sliced. Took 0.13129091262817383 seconds
Function Begins: Loading fZ from 202215 to 404430
Loading Data...
Data Loaded. Took 0.11772394180297852 seconds
Sorting Data...
Data Sorted. Took 0.1268160343170166 seconds
Slicing Data...
Data Sliced. Took 0.12687301635742188 seconds
Function Begins: Loading fPolX from 202215 to 404430
Lo

In [62]:
final_array.sum(axis=0)

array([2022146, 1030548,  839157,  853834,  821006,  826948,  813845,
        819640])

So at least for one batch we got:  
20224365, 26489425, 8392283, 23172857, 8211833, 14059395
for the data labels:   
- len(x_mm), 
- len(time_window_coincidences), 
- true_in_energy_filtered_count, 
- total_in_energy_filtered_count, 
- true_in_polarization_filtered_count, 
- total_in_polarization_filtered_count 